Q1

The tutorial on MAB was taken

Installing tf-agents and importing the necessary packages

In [ ]:
pip install tf-agents

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.4/624.4 KB 44.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 47.8 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.23.0-py3-none-any.whl size=697661 sha256=8d3c874552e7e4d7a9e30769e6b79016c8f4546262fd58d844667665c9c3e2ee
  Stored in directory: /root/.cache/pip/wheels/e7/2f/ab/68bf956c5dde73c1856d981e54292cf58385fb60bca10b7acd
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2


In [ ]:
import numpy as np
import tensorflow as tf
import abc

from tf_agents.agents import tf_agent
from tf_agents.drivers import driver
from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.policies import tf_policy
from tf_agents.specs import array_spec
from tf_agents.specs import tensor_spec
from tf_agents.trajectories import time_step as ts
from tf_agents.trajectories import trajectory
from tf_agents.trajectories import policy_step

nest = tf.nest

Q2 Exercise-1

In [ ]:
class BanditPyEnvironment(py_environment.PyEnvironment):

  def __init__(self, observation_spec, action_spec):
    self._observation_spec = observation_spec
    self._action_spec = action_spec
    super(BanditPyEnvironment, self).__init__()

  # Helper functions.
  def action_spec(self):
    return self._action_spec

  def observation_spec(self):
    return self._observation_spec

  def _empty_observation(self):
    return tf.nest.map_structure(lambda x: np.zeros(x.shape, x.dtype),
                                 self.observation_spec())

  # These two functions below should not be overridden by subclasses.
  def _reset(self):
    """Returns a time step containing an observation."""
    return ts.restart(self._observe(), batch_size=self.batch_size)

  def _step(self, action):
    """Returns a time step containing the reward for the action taken."""
    reward = self._apply_action(action)
    return ts.termination(self._observe(), reward)

  # These two functions below are to be implemented in subclasses.
  @abc.abstractmethod
  def _observe(self):
    """Returns an observation."""

  @abc.abstractmethod
  def _apply_action(self, action):
    """Applies `action` to the Environment and returns the corresponding reward.
    """

In [ ]:
class SimplePyEnvironment(BanditPyEnvironment):

  def __init__(self):
    action_spec = array_spec.BoundedArraySpec(
        shape=(), dtype=np.int32, minimum=0, maximum=2, name='action')
    observation_spec = array_spec.BoundedArraySpec(
        shape=(1,), dtype=np.int32, minimum=-5, maximum=5, name='observation')
    super(SimplePyEnvironment, self).__init__(observation_spec, action_spec)

  def _observe(self):
    self._observation = np.random.randint(-2, 3, (1,), dtype='int32')
    return self._observation

  def _apply_action(self, action):
    return action * self._observation

In [ ]:
environment = SimplePyEnvironment()
observation = environment.reset().observation
print("observation: %d" % observation)

action = 2

net_reward = 0

for i in range (0,50):
  print("\n")
  print("iteration %d" % i)
  print("action: %d" % action)
  reward = environment.step(action).reward
  net_reward += reward
  print("reward: %f" % reward)

print("Net reward after 50 actions: %d" % net_reward)

observation: 1


iteration 0
action: 2
reward: 2.000000


iteration 1
action: 2
reward: 2.000000


iteration 2
action: 2
reward: 0.000000


iteration 3
action: 2
reward: 4.000000


iteration 4
action: 2
reward: 4.000000


iteration 5
action: 2
reward: -2.000000


iteration 6
action: 2
reward: -2.000000


iteration 7
action: 2
reward: 2.000000


iteration 8
action: 2
reward: 4.000000


iteration 9
action: 2
reward: 2.000000


iteration 10
action: 2
reward: -4.000000


iteration 11
action: 2
reward: 2.000000


iteration 12
action: 2
reward: 0.000000


iteration 13
action: 2
reward: 4.000000


iteration 14
action: 2
reward: -4.000000


iteration 15
action: 2
reward: 0.000000


iteration 16
action: 2
reward: -4.000000


iteration 17
action: 2
reward: 4.000000


iteration 18
action: 2
reward: 2.000000


iteration 19
action: 2
reward: 4.000000


iteration 20
action: 2
reward: -4.000000


iteration 21
action: 2
reward: -4.000000


iteration 22
action: 2
reward: -4.000000


iteration 23
action

Therefore the net reward after 50 action is as above and it changes every time we run the experiment

In [ ]:
class SignPolicy(tf_policy.TFPolicy):
  def __init__(self):
    observation_spec = tensor_spec.BoundedTensorSpec(
        shape=(1,), dtype=tf.int32, minimum=-2, maximum=2)
    time_step_spec = ts.time_step_spec(observation_spec)

    action_spec = tensor_spec.BoundedTensorSpec(
        shape=(), dtype=tf.int32, minimum=0, maximum=2)

    super(SignPolicy, self).__init__(time_step_spec=time_step_spec,
                                     action_spec=action_spec)
  def _distribution(self, time_step):
    pass

  def _variables(self):
    return ()

  def _action(self, time_step, policy_state, seed):
    observation_sign = tf.cast(tf.sign(time_step.observation[0]), dtype=tf.int32)
    action = observation_sign + 1
    return policy_step.PolicyStep(action, policy_state)

In [ ]:
tf_environment = tf_py_environment.TFPyEnvironment(environment)

In [ ]:
sign_policy = SignPolicy()

current_time_step = tf_environment.reset()
print('Observation:')
print (current_time_step.observation)
action = sign_policy.action(current_time_step).action
print('Action:')
print (action)
reward = tf_environment.step(action).reward
print('Reward:')
print(reward)

Observation:
tf.Tensor([[0]], shape=(1, 1), dtype=int32)
Action:
tf.Tensor([1], shape=(1,), dtype=int32)
Reward:
tf.Tensor([[0.]], shape=(1, 1), dtype=float32)


In [ ]:
step = tf_environment.reset()
action = 1
next_step = tf_environment.step(action)

net_reward = 0

for i in range (0,50):
  print("\n")
  print("iteration %d" % i)
  reward = next_step.reward
  next_observation = next_step.observation
  print("Reward: ")
  print(reward)
  print("Next observation:")
  print(next_observation)

print("Net reward after 50 actions: %d" % net_reward)



iteration 0
Reward: 
tf.Tensor([[2.]], shape=(1, 1), dtype=float32)
Next observation:
tf.Tensor([[0]], shape=(1, 1), dtype=int32)


iteration 1
Reward: 
tf.Tensor([[2.]], shape=(1, 1), dtype=float32)
Next observation:
tf.Tensor([[0]], shape=(1, 1), dtype=int32)


iteration 2
Reward: 
tf.Tensor([[2.]], shape=(1, 1), dtype=float32)
Next observation:
tf.Tensor([[0]], shape=(1, 1), dtype=int32)


iteration 3
Reward: 
tf.Tensor([[2.]], shape=(1, 1), dtype=float32)
Next observation:
tf.Tensor([[0]], shape=(1, 1), dtype=int32)


iteration 4
Reward: 
tf.Tensor([[2.]], shape=(1, 1), dtype=float32)
Next observation:
tf.Tensor([[0]], shape=(1, 1), dtype=int32)


iteration 5
Reward: 
tf.Tensor([[2.]], shape=(1, 1), dtype=float32)
Next observation:
tf.Tensor([[0]], shape=(1, 1), dtype=int32)


iteration 6
Reward: 
tf.Tensor([[2.]], shape=(1, 1), dtype=float32)
Next observation:
tf.Tensor([[0]], shape=(1, 1), dtype=int32)


iteration 7
Reward: 
tf.Tensor([[2.]], shape=(1, 1), dtype=float32)
Next o

Q3 Exercise-2

a)

In [ ]:
class RewardEnv(py_environment.PyEnvironment):
  def __init__(self, reward_sign):
    self._observation_spec = array_spec.BoundedArraySpec(
        shape=(), dtype=np.int32, minimum=-5, maximum=5, name='observation')
    self._action_spec = array_spec.BoundedArraySpec(
        shape=(), dtype=np.int32, minimum=0, maximum=2, name='action')
    self._episode_ended = False
    self._observation = None
    self._reward = None
    self._reward_sign = reward_sign

  def action_spec(self):
    return self._action_spec

  def observation_spec(self):
    return self._observation_spec

  def _reset(self):
    self._episode_ended = False
    self._observation = np.random.randint(low=-5, high=6)
    self._reward = 0
    return ts.restart(np.array(self._observation, dtype=np.int32))

  def _step(self, action):
    if self._episode_ended:
      return self.reset()

    if self._reward_sign == 'original':
      self._reward = self._observation * action
    else:
      self._reward = -self._observation * action

    self._episode_ended = True
    return ts.termination(np.array(self._observation, dtype=np.int32), reward=self._reward)


b)

In [ ]:
class Policy:
  def __init__(self):
    self._state = 'unknown'

  def get_action(self, observation):
    if self._state == 'unknown':
      if observation >= 0:
        self._state = 'original'
        return 2
      else:
        self._state = 'flipped'
        return 0
    elif self._state == 'original':
      return 2
    else:
      return 0

c)

In [ ]:
class Agent:
  def __init__(self):
    self._policy = Policy()

  def update_policy(self, reward_sign):
    if reward_sign == 'original':
      self._policy._state = 'original'
    else:
      self._policy._state = 'flipped'

  def get_action(self, observation):
    return self._policy.get_action(observation)

output from each case

Original Environment (reward = observation * action)

In [ ]:
reward_env = RewardEnv(reward_sign='original')
agent = Agent()

total_reward = 0
for i in range(50):
    observation = reward_env.reset().observation
    agent.update_policy('original')
    action = agent.get_action(observation)
    time_step = reward_env.step(action)
    total_reward += time_step.reward
print("Total reward: ", total_reward)

Total reward:  28.0


The total reward should be positive since the optimal policy will result in positive rewards for positive observations and negative rewards for negative observations.

Flipped Environment (reward = -observation * action)

In [ ]:
reward_env = RewardEnv(reward_sign='flipped')
agent = Agent()

total_reward = 0
for i in range(50):
    observation = reward_env.reset().observation
    agent.update_policy('flipped')
    action = agent.get_action(observation)
    time_step = reward_env.step(action)
    total_reward += time_step.reward
print("Total reward: ", total_reward)

Total reward:  0.0


This defines an environment where the reward is always 0, regardless of the action and observation. Therefore, the total reward in this environment should always be 0.